Idenya adalah
1. jika topics belum ada,maka ambil 500 post popular pertama di 7 hari kebelakang lalu di clustering 
2. return ke UI
3. hasil dari point pertama di masukan ke ES in background
---
4. jika sudah ada, maka API hanya akan send hasil yg ada di ES saja, tidak perlu bikin ulang
---
5. terdapat JOB yg akan melakukan point pertama berulang ulang agar data semakin kaya

Contoh Output API


{
        "unified_issue": "Banking and Financial Institution Activities",
        "description": "Performance, strategies, and business activities of various banks and financial institutions in Indonesia.",
        "list_issue": [
            "Microfinance bank association members' performance",
            "Morning Gymnastics at Bank Indonesia Kediri"
        ],
        "total_posts": 291,
        "viral_score": 71.81342340121046,
        "reach_score": 800.3937775007216,
        "positive": 149,
        "negative": 7,
        "neutral": 135,
        "share_of_voice": 5.752124925874679
    },

# untuk topics yg baru dibuat

In [4]:
#check apakah project sudah ada?
from elasticsearch import Elasticsearch, helpers
from typing import List, Dict, Any, Optional
import os, re
import pandas as pd
from utils.gemini import call_gemini
from utils.list_of_mentions import get_mentions
from utils.topics.es_operations import upsert_documents
import uuid
from datetime import datetime, timedelta
ES_HOST = os.getenv('ES_HOST','http://34.101.178.71:9200')
ES_USERNAME = os.getenv('ES_USERNAME','elastic')
ES_PASSWORD = os.getenv('ES_PASSWORD','elasticpassword')
# Inisialisasi client
es = Elasticsearch(hosts=ES_HOST, http_auth = (ES_USERNAME, ES_PASSWORD))

def get_data_topics(project_name):
    query_body = {
      "query": {
        "bool": {
            "must":{
                "match":{
                    "project_name":project_name
                }

            }
        }
      }
    }

    data = es.search(
        index = index_name,
        body = query_body,
        size = 10000
    )

    topics_overview = [i['_source'] for i in data['hits']['hits']]
    return topics_overview

def create_uuid(keyword: str) -> str:
    """Generate UUID for a given keyword"""
    namespace = uuid.NAMESPACE_DNS
    return str(uuid.uuid5(namespace, keyword))

def ingest_topic(results, project_name, es = None) -> None:
    """Ingest topic results into Elasticsearch"""
    if es is None:
        es = get_elasticsearch_client()

    df = pd.DataFrame(results)
    df['project_name'] = project_name
    df['index'] = range(df.shape[0])
    df['uuid'] = df.apply(lambda s: create_uuid(f"{s['project_name']},{s['unified_issue']}" if len(s['list_issue'])<100\
                                                else f"{s['project_name']},{s['unified_issue']},{s['index']}"), axis=1)

    data_ingest = df[['unified_issue', 'description', 'list_issue', 'uuid', 'project_name']].to_dict(orient='records')

    updated, created, errors = upsert_documents(
        es,
        data_ingest,
        "topics_overview",
        id_field="uuid",
        fields_to_update=None,
        chunk_size=1000
    )

    print(f"Results: {updated} documents updated, {created} documents created, {len(errors)} errors")
    
def split_rows_by_list_length(df, column='list_issue', max_len=100):
    """
    Membelah baris jika panjang list dalam kolom tertentu melebihi max_len.

    Args:
        df (pd.DataFrame): DataFrame dengan kolom list.
        column (str): Nama kolom yang berisi list.
        max_len (int): Jumlah maksimum elemen per baris.

    Returns:
        pd.DataFrame: DataFrame baru dengan baris terbelah.
    """
    new_rows = []

    for _, row in df.iterrows():
        items = row[column]
        # Membagi list menjadi potongan dengan panjang maksimal max_len
        chunks = [items[i:i + max_len] for i in range(0, len(items), max_len)]
        for chunk in chunks:
            new_row = row.copy()
            new_row[column] = chunk
            new_rows.append(new_row)

    return pd.DataFrame(new_rows).reset_index(drop=True)    

def get_date_range(date_filter="last 30 days", custom_start_date=None, custom_end_date=None):

    today = datetime.now().date()
    
    if date_filter == "custom" and custom_start_date and custom_end_date:
        return custom_start_date, custom_end_date
    
    if date_filter == "yesterday":
        yesterday = today - timedelta(days=1)
        return yesterday.strftime("%Y-%m-%d"), yesterday.strftime("%Y-%m-%d")
    
    elif date_filter == "this week":
        start_of_week = today - timedelta(days=today.weekday())
        return start_of_week.strftime("%Y-%m-%d"), today.strftime("%Y-%m-%d")

    elif date_filter == "last 14 days":
        return (today - timedelta(days=14)).strftime("%Y-%m-%d"), today.strftime("%Y-%m-%d")

    elif date_filter == "last 7 days":
        return (today - timedelta(days=7)).strftime("%Y-%m-%d"), today.strftime("%Y-%m-%d")
    
    elif date_filter == "last 30 days":
        return (today - timedelta(days=30)).strftime("%Y-%m-%d"), today.strftime("%Y-%m-%d")
    
    elif date_filter == "last 3 months":
        return (today - timedelta(days=90)).strftime("%Y-%m-%d"), today.strftime("%Y-%m-%d")
    
    elif date_filter == "this year":
        return f"{today.year}-01-01", today.strftime("%Y-%m-%d")
    
    elif date_filter == "last year":
        return f"{today.year-1}-01-01", f"{today.year-1}-12-31"
    
    else:  # "all time" or fallback
        return "2000-01-01", today.strftime("%Y-%m-%d")
    
def regenerate_topics(suggestion_unified_issue,df_issue):
    #call gemini untuk clustering
    issue_list = []
    for idx, row in df_issue\
                    .sort_values(['total_post','negative_posts','total_reach_score'],
                                ascending = [False,False,False])[:100].iterrows():
        issue_list.append({"id": row['index'], "issue": row['issue']})

    prompt = f"""
    You are a Social Media Analysis Expert specializing in thematic clustering of issues.

    # TASK
    Analyze and group the following list of social media issues into meaningful thematic clusters.

    # IMPORTANT INSTRUCTIONS
    1. Each issue ID must belong to EXACTLY ONE group (mutually exclusive).
    2. Avoid any overlap of issues between groups.
    3. Focus on identifying the main topic or theme of each issue.
    4. Create a unified_issue name that is **clear, specific, and truly represents the essence** of the grouped issues.
    5. Avoid using overly generic group names like "General Issues" or "Various Topics."
    6. Provide an informative and comprehensive description for each unified_issue that:
       - Accurately summarizes its main theme,
       - Highlights prevalent sentiment(s) (e.g., positive, negative, neutral),
       - Mentions any notable patterns, concerns, or emotional tones observed,
       - Includes relevant contextual details or recurring keywords if applicable.
    7. If an issue is not related to the topic of "{project_name}", group it under the name **"Other"** and provide a description that clearly reflects the non-relevance to "{project_name}."
    8. Use the following suggestion list of possible unified_issue names and descriptions as references. If any suggestion fits well for a group of issues, feel free to use that unified_issue name and description exactly as provided.
    
    ## SUGGESTED UNIFIED_ISSUES (unified_issue and description)
    ```
    {suggestion_unified_issue}
    ```

    # LIST OF ISSUES
    ```
    {issue_list}
    ```

    # OUTPUT FORMAT
    Return the grouped results strictly in the following JSON format:
    ```
    [
    {{
    "unified_issue": "Name of Grouped Issue 1",
    "description": "Concise description summarizing the main theme of Group 1",
    "list_issue_id": [1, 5, 10, 15]
    }},
    {{
    "unified_issue": "Name of Grouped Issue 2",
    "description": "Concise description summarizing the main theme of Group 2",
    "list_issue_id": [2, 6, 11, 16]
    }},
    ...
    ]
    ```

    # QUALITY PARAMETERS
    - Group based on semantic similarity of topics and keywords.
    - Choose a **specific** and **meaningful** name for each unified_issue that reflects the core topic clearly.

    # HARD RULES:
    - Return the output ONLY in pure JSON format, without any explanation or additional comments.
    - The unified_issue name and description must be in English.
    """

    centrality = call_gemini(prompt)
    try:
        data_center = pd.DataFrame(eval(re.findall(r'\[.*\]', centrality, flags=re.I|re.S)[0]))
    except:
        unified_issue = [i.strip('\n ,"') for i in re.findall(r'unified_issue[\"\,\s\:]+(.*?)description',centrality, flags=re.I|re.S)]
        description = [i.strip('\n ,"') for i in re.findall(r'description[\"\,\s\:]+(.*?)list_issue_id',centrality, flags=re.I|re.S)]
        list_issue_id = [eval(i.strip('\n ,"')) for i in re.findall(r'list_issue_id[\"\,\s\:]+(.*?)\}',centrality, flags=re.I|re.S)]

        data = []
        for u,d,l in zip(unified_issue, description, list_issue_id):
            data.append({'unified_issue':u, 'description':d,'list_issue_id':l})
        data_center = pd.DataFrame(data)


    df_exploded = data_center.explode('list_issue_id')
    df_exploded = df_exploded.rename(columns={"list_issue_id": "index"})

    df_merged = pd.merge(
        df_exploded[['index', 'unified_issue', 'description']],
        df_issue,
        on='index',
        how='left'
    )

    df_final = df_merged.groupby(['unified_issue', 'description']).agg(
        list_issue=('issue', list),
        total_posts=('total_post', 'sum'),
        viral_score=('total_viral_score', 'sum'),
        reach_score=('total_reach_score', 'sum'),
        positive=('positive_posts', 'sum'),
        negative=('negative_posts', 'sum'),
        neutral=('neutral_posts', 'sum')
    ).reset_index()

    df_final['share_of_voice'] = (df_final['total_posts']/df_final['total_posts'].sum())*100
    df_final = df_final.sort_values('share_of_voice', ascending=False)

    return df_final

In [5]:
def get_data_issue(owner_id: Optional[str] = 1,
    project_name: Optional[str] = None,
    es: Optional[Elasticsearch] = None,
    keywords: Optional[str] = [],
    search_keyword: Optional[str] = [],
    search_exact_phrases: bool = False,
    case_sensitive: bool = False,
    sentiment: Optional[str] = ['positive','negative','neutral'],
    start_date: Optional[str] = None,
    end_date: Optional[str] = None,
    date_filter: str = "last 30 days",
    custom_start_date: Optional[str] = None,
    custom_end_date: Optional[str] = None,
    channels: Optional[List[str]] = None,
    importance: str = "all mentions",
    influence_score_min: Optional[float] = 0,
    influence_score_max: Optional[float] = 1000,
    region: Optional[str] = [],
    language: Optional[str] = [],
    domain: Optional[str] = [],
    limit=1000
              ):

    list_keywords = [
                        {
                          "match": {
                            "post_caption": {
                              "query": i,
                              "operator": "AND"
                            }
                          }
                        } for i in keywords
                      ]
    list_keywords.extend([ {
                          "match": {
                            "issue": {
                              "query": i,
                              "operator": "AND"
                            }
                          }
                        } for i in keywords

    ])
    

    if not start_date or not end_date:
        start_date, end_date = get_date_range(
            date_filter=date_filter,
            custom_start_date=custom_start_date,
            custom_end_date=custom_end_date
        )
    
    
    query = {
      "size": 0,
      "query": {
        "bool": {
          "must": [
            {
              "range": {
                "post_created_at": {
                    "gte": start_date,
                    "lte": end_date
                }
              }
            },
            {
              "bool": {
                "must": [
                  {
                    "bool": {
                      "should": list_keywords,



                      "minimum_should_match": 1
                    }
                  },
                  {
                    "bool": {
                      "should": [
                        {
                          "match": {
                            "post_caption": {
                              "query": f"{i}",
                              "operator": "AND"
                            }
                          }
                        } for i in search_keyword
                      ],
                      "minimum_should_match": 1
                    }
                  }
                ]
              }
            },
            {
                
            "bool":{
                "must_not":{
                    "match":{
                        "issue":"Not Specified"
                        
                    }
                    
                    
                }
                
            }
                
                
            }
          ],
          "filter": [
            {
              "terms": {
                "sentiment": sentiment
              }
            },
            {
              "range": {
                "influence_score": {
                  "gte": influence_score_min,
                  "lte": influence_score_max
                }
              }
            },
            {
              "bool": {
                "should": [
                  {
                    "wildcard": {
                      "region": f"*{i}*"
                    }
                  } for i in region
                ],
                "minimum_should_match": 1
              }
            },
            {
              "bool": {
                "should": [
                  {
                    "wildcard": {
                      "language": f"*{i}*"
                    }
                  } for i in language
                ],
                "minimum_should_match": 1
              }
            },
            {
              "bool": {
                "should": [
                  {
                    "wildcard": {
                      "link_post": f"*{i}*"
                    }
                  }
                    for i in domain
                ],
                "minimum_should_match": 1
              }
            },
            {
              "bool": {
                "must": [
                  {
                    "exists": {
                      "field": "viral_score"
                    }
                  },
                  {
                    "exists": {
                      "field": "sentiment"
                    }
                  }
                ]
              }
            }
          ]
        }
      },
      "from": 0,
      "aggs": {
        "by_issue": {
          "terms": {
            "field": "issue.keyword",
            "size": limit,
            "order": {
              "_count": "desc"
            }
          },
          "aggs": {
            "total_viral_score": {
              "sum": { "field": "viral_score" }
            },
            "total_reach_score": {
              "sum": { "field": "reach_score" }
            },
            "total_post": {
              "value_count": { "field": "issue.keyword" }
            },
            "positive_posts": {
              "filter": { "term": { "sentiment": "positive" } }
            },
            "negative_posts": {
              "filter": { "term": { "sentiment": "negative" } }
            },
            "neutral_posts": {
              "filter": { "term": { "sentiment": "neutral" } }
            },
            "sample_posts": {
              "top_hits": {
                "size": 2,
                "_source": {
                  "includes": [
                    "post_caption",
                      "link_post"
                  ]
                }
              }
            }
          }
        }
      }
    }
    

    if not channels:
        channels = ['tiktok','instagram','news','reddit','facebook','twitter','linkedin','youtube']
    
    index = ','.join([i+'_data' for i in channels])
    hasil = es.search(index = index,
             body = query)
    
    buckets = hasil['aggregations']['by_issue']['buckets']
    
    data = []
    for bucket in buckets:
        sample_posts_list = []
        for hit in bucket['sample_posts']['hits']['hits']:
            post = hit['_source']
            sample_posts_list.append({
                "post_caption": post.get("post_caption", ""),
                "link_post": post.get("link_post", "")
            })

        data.append({
            "issue": bucket['key'],
            "total_viral_score": bucket['total_viral_score']['value'],
            "total_reach_score": bucket['total_reach_score']['value'],
            "total_post": bucket['total_post']['value'],
            "positive_posts": bucket['positive_posts']['doc_count'],
            "negative_posts": bucket['negative_posts']['doc_count'],
            "neutral_posts": bucket['neutral_posts']['doc_count'],
            "sample_posts": sample_posts_list
        })

    # Buat DataFrame
    df_issue = pd.DataFrame(data).reset_index()
    return df_issue    

def new_topics(owner_id: Optional[str] = 1,
    project_name: Optional[str] = None,
    es: Optional[Elasticsearch] = None,
    keywords: Optional[str] = [],
    search_keyword: Optional[str] = [],
    search_exact_phrases: bool = False,
    case_sensitive: bool = False,
    sentiment: Optional[str] = ['positive','negative','neutral'],
    start_date: Optional[str] = None,
    end_date: Optional[str] = None,
    date_filter: str = "last 30 days",
    custom_start_date: Optional[str] = None,
    custom_end_date: Optional[str] = None,
    channels: Optional[List[str]] = None,
    importance: str = "all mentions",
    influence_score_min: Optional[float] = 0,
    influence_score_max: Optional[float] = 1000,
    region: Optional[str] = [],
    language: Optional[str] = [],
    domain: Optional[str] = []
              ):

    # Buat DataFrame
    df_issue = get_data_issue(
        owner_id = owner_id,
        es = es,
        project_name = project_name,
        keywords=keywords,
        search_keyword=search_keyword,
        search_exact_phrases=search_exact_phrases,
        case_sensitive=case_sensitive,
        sentiment=sentiment,
        start_date=start_date,
        end_date=end_date,
        date_filter=date_filter,
        custom_start_date=custom_start_date,
        custom_end_date=custom_end_date,
        channels=channels,
        importance=importance,
        influence_score_min=influence_score_min,
        influence_score_max=influence_score_max,
        region=region,
        language=language,
        domain=domain
    
    )
    
    #call gemini untuk clustering
    issue_list = []
    for idx, row in df_issue\
                    .sort_values(['total_post','negative_posts','total_reach_score'],
                                ascending = [False,False,False])[:50].iterrows():
        issue_list.append({"id": row['index'], "issue": row['issue']})

    prompt = f"""
    You are a Social Media Analysis Expert specializing in thematic clustering of issues.

    # TASK
    Analyze and group the following list of social media issues into meaningful thematic clusters.

    # IMPORTANT INSTRUCTIONS
    1. Each issue ID must belong to EXACTLY ONE group (mutually exclusive).
    2. Avoid any overlap of issues between groups.
    3. Focus on identifying the main topic or theme of each issue.
    4. Create a unified_issue name that is **clear, specific, and truly represents the essence** of the grouped issues.
    5. Avoid using overly generic group names like "General Issues" or "Various Topics."
    6. Provide an informative and comprehensive description for each unified_issue that:
       - Accurately summarizes its main theme,
       - Highlights prevalent sentiment(s) (e.g., positive, negative, neutral),
       - Mentions any notable patterns, concerns, or emotional tones observed,
       - Includes relevant contextual details or recurring keywords if applicable.

    # LIST OF ISSUES
    ```
    {issue_list}
    ```

    # OUTPUT FORMAT
    Return the grouped results strictly in the following JSON format:
    ```
    [
    {{
    "unified_issue": "Name of Grouped Issue 1",
    "description": "Concise description summarizing the main theme of Group 1",
    "list_issue_id": [1, 5, 10, 15]
    }},
    {{
    "unified_issue": "Name of Grouped Issue 2",
    "description": "Concise description summarizing the main theme of Group 2",
    "list_issue_id": [2, 6, 11, 16]
    }},
    ...
    ]
    ```

    # QUALITY PARAMETERS
    - Group based on semantic similarity of topics and keywords.
    - Choose a **specific** and **meaningful** name for each unified_issue that reflects the core topic clearly.

    # HARD RULES:
    - Return the output ONLY in pure JSON format, without any explanation or additional comments.
    - The unified_issue name and description must be in English.
    """

    centrality = call_gemini(prompt)
    try:
        data_center = pd.DataFrame(eval(re.findall(r'\[.*\]', centrality, flags=re.I|re.S)[0]))
    except:
        unified_issue = [i.strip('\n ,"') for i in re.findall(r'unified_issue[\"\,\s\:]+(.*?)description',centrality, flags=re.I|re.S)]
        description = [i.strip('\n ,"') for i in re.findall(r'description[\"\,\s\:]+(.*?)list_issue_id',centrality, flags=re.I|re.S)]
        list_issue_id = [eval(i.strip('\n ,"')) for i in re.findall(r'list_issue_id[\"\,\s\:]+(.*?)\}',centrality, flags=re.I|re.S)]

        data = []
        for u,d,l in zip(unified_issue, description, list_issue_id):
            data.append({'unified_issue':u, 'description':d,'list_issue_id':l})
        data_center = pd.DataFrame(data)

        
    df_exploded = data_center.explode('list_issue_id')
    df_exploded = df_exploded.rename(columns={"list_issue_id": "index"})

    df_merged = pd.merge(
        df_exploded[['index', 'unified_issue', 'description']],
        df_issue,
        on='index',
        how='left'
    )

    df_final = df_merged.groupby(['unified_issue', 'description']).agg(
        list_issue=('issue', list),
        total_posts=('total_post', 'sum'),
        viral_score=('total_viral_score', 'sum'),
        reach_score=('total_reach_score', 'sum'),
        positive=('positive_posts', 'sum'),
        negative=('negative_posts', 'sum'),
        neutral=('neutral_posts', 'sum')
    ).reset_index()

    df_final['share_of_voice'] = (df_final['total_posts']/df_final['total_posts'].sum())*100
    df_final = df_final.sort_values('share_of_voice', ascending=False)

    results = df_final.to_dict(orient = 'records')
    
    #--------------- sisa df_final di background ----------------
    print('DIBACKGROUND')
    ingest_topic(results, project_name, es = es)
    
    loop=2
    for i in range(loop):
        suggestion_unified_issue = df_final[['unified_issue','description']].to_dict(orient = 'records')
        df_issue = df_issue[~df_issue['issue'].isin([j for i in df_final['list_issue'] for j in i])]
        if df_issue.empty:
            break
        df_final2 = regenerate_topics(suggestion_unified_issue,df_issue)
        df_final = pd.concat([df_final, df_final2])
        df_final = df_final.groupby(['unified_issue']).agg(
                description=('description', 'max'),
                list_issue=('list_issue', lambda s: [j for i in s for j in i if not pd.isna(j)]),
                total_posts=('total_posts', 'sum'),
                viral_score=('viral_score', 'sum'),
                reach_score=('reach_score', 'sum'),
                positive=('positive', 'sum'),
                negative=('negative', 'sum'),
                neutral=('neutral', 'sum')).reset_index()
        df_final['share_of_voice'] = (df_final['total_posts']/df_final['total_posts'].sum())*100
        df_final = split_rows_by_list_length(df_final)

        print(df_final.shape, df_final['unified_issue'].nunique(),len(set([j for i in df_final['list_issue'] for j in i])))
        results = df_final.to_dict(orient = 'records')
        ingest_topic(results, project_name,es = es)
    # --------------------- --------------------------------------
    
    return results

In [6]:
index_name = "topics_overview"
project_name = "danantara"
keywords = ['danantara',"investasi indonesia"]
owner_id = 5
search_keyword = []
domain = []
language = []
region = []
influence_score_min= 0
influence_score_max= 1000
sentiment = [
    "positive",
    "negative",
    "neutral"
  ]
start_date = None
end_date = None
date_filter = 'last 7 days'
custom_start_date=None
custom_end_date = None
channels = None

In [7]:
#apakah topics overview sudah ada?
#jika sudah maka return else, olah per 500 data popular
topics_overview = get_data_topics(project_name)
if not topics_overview:
    print('belum ada')
    


INFO:elastic_transport.transport:POST http://34.101.178.71:9200/topics_overview/_search [status:200 duration:0.090s]


In [ ]:
new_topics(project_name = project_name,
           keywords = keywords,
           es = es)

# misalnya nyari di date lain

In [20]:
index_name = "topics_overview"
project_name = "danantara"
keywords = ['danantara',"investasi indonesia"]
owner_id = 5
search_keyword = []
domain = []
language = []
region = []
influence_score_min= 0
influence_score_max= 1000
sentiment = [
    "positive",
    "negative",
    "neutral"
  ]
start_date = None
end_date = None
date_filter = 'last 7 days'
custom_start_date=None
custom_end_date = None
channels = None

In [34]:
#apakah topics overview sudah ada?
#jika sudah maka return else, olah per 500 data popular
topics_overview = get_data_topics(project_name)
if not topics_overview:
    print('belum ada')
    hasil = new_topics(project_name = project_name,
               keywords = keywords,
               es = es)
else:
    print('sudah ada')
    

INFO:elastic_transport.transport:POST http://34.101.178.71:9200/topics_overview/_search [status:200 duration:0.136s]


sudah ada


In [35]:

df_issue = get_data_issue(
    es = es,
    project_name = project_name,
    keywords=keywords,
    search_keyword=search_keyword,
    sentiment=sentiment,
    start_date=start_date,
    end_date=end_date,
    date_filter=date_filter,
    custom_start_date=custom_start_date,
    custom_end_date=custom_end_date,
    channels=channels,
    influence_score_min=influence_score_min,
    influence_score_max=influence_score_max,
    region=region,
    language=language,
    domain=domain,
    limit=500
)
#ini yg direturn
data_center = pd.DataFrame(topics_overview)
data_center

df_exploded = data_center.explode('list_issue')
df_exploded = df_exploded.rename(columns={"list_issue": "issue"})

df_merged = pd.merge(
    df_exploded[['issue', 'unified_issue', 'description']],
    df_issue,
    on='issue'
)

df_final = df_merged.groupby(['unified_issue', 'description']).agg(
    list_issue=('issue', list),
    total_posts=('total_post', 'sum'),
    viral_score=('total_viral_score', 'sum'),
    reach_score=('total_reach_score', 'sum'),
    positive=('positive_posts', 'sum'),
    negative=('negative_posts', 'sum'),
    neutral=('neutral_posts', 'sum')
).reset_index()

df_final['share_of_voice'] = (df_final['total_posts']/df_final['total_posts'].sum())*100
df_final = df_final.sort_values('share_of_voice', ascending=False)


results = df_final.to_dict(orient = 'records')
df_final

INFO:elastic_transport.transport:POST http://34.101.178.71:9200/tiktok_data,instagram_data,news_data,reddit_data,facebook_data,twitter_data,linkedin_data,youtube_data/_search [status:200 duration:2.169s]


,unified_issue,description,list_issue,total_posts,viral_score,reach_score,positive,negative,neutral,share_of_voice
75,Other,This cluster includes issues that are not dire...,"[BPKH manages hajj funds effectively, BPKH man...",289,55.277300,1197.214998,70,27,192,42.128280
58,Indonesian Economic Performance and Outlook,This cluster encompasses discussions and repor...,"[IHSG (Jakarta Composite Index) movement, Indo...",21,4.626327,48.000000,3,7,11,3.061224
28,Danantara Initiatives and Support,"This cluster focuses on Danantara's role, stra...","[Will Danantara become a successful force?, Da...",19,9.511562,137.681667,3,0,16,2.769679
39,Economic and Financial Market Updates,This cluster covers various aspects of Indones...,"[IHSG (Jakarta Composite Index) movement, Indo...",17,4.135826,73.700000,2,5,10,2.478134
56,Government and State Official Activities,This cluster includes activities and statement...,"[Farry Francis, President's envoy, Finance Min...",15,5.252781,64.310000,0,0,15,2.186589
...,...,...,...,...,...,...,...,...,...,...
24,Corporate Social Responsibility and Investment...,This cluster includes news about companies eng...,[CNBC Indonesia presents investment ideas],1,0.092715,7.700000,0,0,1,0.145773
33,E-commerce and Logistics Challenges,This cluster focuses on challenges faced by e-...,[E-commerce players complain about high shipping],1,0.500000,1.550000,0,1,0,0.145773
55,Government Policies and Social Programs,This cluster includes discussions related to g...,[Free Nutritious Meal Program Prabowo-Gibran],1,0.636746,4.505000,0,0,1,0.145773
61,Insurance Industry Pressure,This cluster highlights the considerable press...,[Considerable pressure on the insurance industry],1,0.400000,0.800000,0,1,0,0.145773


In [36]:
df_final['total_posts'].sum()

686

In [30]:
print('--------------in background --------------')
data_center

--------------in background --------------


,unified_issue,description,list_issue,uuid,project_name
0,Economic Challenges and Discussions,This cluster focuses on economic challenges an...,"[Kadin discusses economic challenges, Kadin In...",6ee868af-d771-5cd5-bc72-0205a0e993b1,danantara
1,Economic Discussions and Government Policies,This cluster includes discussions from key gov...,"[Minister of Energy's statement, Minister of F...",311e05e0-5c22-52c9-8baf-a01ca0241746,danantara
2,Efforts to achieve food self-sufficiency,This cluster focuses on efforts to achieve foo...,"[Efforts to achieve food self-sufficiency, Coo...",0ca51090-fc99-53d8-b8b9-229d6dcb72fc,danantara
3,Financial Institution Initiatives,This cluster focuses on initiatives by financi...,"[Kiwoom Sekuritas daily research, Bank Mandiri...",5ad2018c-a298-54e2-9f68-198b63950499,danantara
4,International Cooperation and Agreements,This cluster focuses on Indonesia's cooperatio...,[Indonesia invites Japan to strengthen green e...,fd74a861-d88c-5c58-86fe-92dd3626e2b4,danantara
...,...,...,...,...,...
84,IHSG and Capital Market Updates,This cluster focuses on updates and prediction...,"[IHSG movement prediction and recommendations,...",5963f3ff-81fa-5cc8-937a-0758715d3b80,danantara
85,International Relations and Visits,This cluster focuses on international relation...,[Australian PM Anthony Albanese's visit to Ind...,4c18161b-69c8-5664-bff4-a4c790d700f9,danantara
86,Investment and Economic Discussions,This cluster centers on investment-related top...,"[CEO of BPI Daya discusses investment, CEO of ...",5a6ee063-c5f9-5e6b-8637-e5a09b294ecc,danantara
87,Other,This cluster includes issues that are not dire...,"[Personal opinion article, Appreciation for lo...",3cf3a3d2-34a8-597a-8eb8-54086fbb24bb,danantara


In [31]:
print(df_final.shape, df_final['unified_issue'].nunique(),len(set([j for i in df_final['list_issue'] for j in i])))

(66, 10) 66 234


In [32]:
#--------------- sisa df_final di background ----------------
print('DIBACKGROUND')
df_final = pd.DataFrame(topics_overview)


loop=2
for i in range(loop):
    suggestion_unified_issue = df_final[['unified_issue','description']].to_dict(orient = 'records')
    df_issue = df_issue[~df_issue['issue'].isin([j for i in df_final['list_issue'] for j in i])]
    if df_issue.empty:
        break
    df_final2 = regenerate_topics(suggestion_unified_issue,df_issue)
    df_final = pd.concat([df_final, df_final2])
    df_final = df_final.groupby(['unified_issue']).agg(
            description=('description', 'max'),
            list_issue=('list_issue', lambda s: [j for i in s for j in i if not pd.isna(j)])).reset_index()
    df_final = split_rows_by_list_length(df_final)

    print(df_final.shape, df_final['unified_issue'].nunique(),len(set([j for i in df_final['list_issue'] for j in i])))
    results = df_final.to_dict(orient = 'records')
    ingest_topic(results, project_name,es = es)
# --------------------- --------------------------------------

DIBACKGROUND


INFO:elastic_transport.transport:POST http://34.101.178.71:9200/topics_overview/_mget?_source=false [status:200 duration:0.042s]
INFO:utils.topics.es_operations:Found 88 existing documents out of 102
INFO:utils.topics.es_operations:Preparing to upsert 102 documents in topics_overview
INFO:utils.topics.es_operations:Processing chunk 1/1 (102 documents, {'update': 88, 'index': 14})


(102, 3) 101 692


INFO:elastic_transport.transport:PUT http://34.101.178.71:9200/_bulk [status:200 duration:0.295s]
INFO:utils.topics.es_operations:Chunk processed successfully: 102 documents
INFO:utils.topics.es_operations:Upsert complete: 88 updated, 14 created, 0 failed


Results: 88 documents updated, 14 documents created, 0 errors


INFO:elastic_transport.transport:POST http://34.101.178.71:9200/topics_overview/_mget?_source=false [status:200 duration:0.058s]
INFO:utils.topics.es_operations:Found 100 existing documents out of 103
INFO:utils.topics.es_operations:Preparing to upsert 103 documents in topics_overview
INFO:utils.topics.es_operations:Processing chunk 1/1 (103 documents, {'update': 100, 'index': 3})


(103, 3) 102 788


INFO:elastic_transport.transport:PUT http://34.101.178.71:9200/_bulk [status:200 duration:0.245s]
INFO:utils.topics.es_operations:Chunk processed successfully: 103 documents
INFO:utils.topics.es_operations:Upsert complete: 100 updated, 3 created, 0 failed


Results: 100 documents updated, 3 documents created, 0 errors


'18'